<a href="https://colab.research.google.com/github/mariotv3/MDL/blob/main/IntroToSeminarCases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install gurobipy

**Code Case 1 Logistics - Team Q1.**

Paulo Andrés de Souza Miranda, Mario Tejera Vicente, Tomás Gutiérrez Morales, Max Viertel Serrano

In [ ]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np
import scipy.stats as sc
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# importing values from excel file
df = pd.read_excel('Railway services-2024.xlsx')
df = df.iloc[:-3]
indexes = df['Trip']
demand = df['Demand(μ)']
line = df['Line']
demand_stdDev = df['Demand(σ)']
arrivalTime = df['Arrival Time']
departureTime = df['Departure Time']
arrivalStation = df['To']
departureStation = df['From']

#licence to run larger models on colab
params = {
"WLSACCESSID": '9f6bda1d-2c0b-49c7-96a8-f172b568e9e4',
"WLSSECRET": '3db7e79a-7091-4079-9dc3-c9c1382b1c4e',
"LICENSEID": 2498504,
}
env = gp.Env(params=params)

**Exercise** 3 - Formulation 1 (Cross-sections + train types)

In [ ]:
# create model
m1 = gp.Model("first formulation ex 3")

# create variables
N = m1.addVars(2, 200, vtype=GRB.INTEGER) #N_{u,c} variables (2 types & 200 cross-sections) (0 = OC & 1 = OH)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict = {}
capacityCoefficients = [620, 420]
capacityDict = {}
lengthCoefficients = [100, 70]
lengthDict = {}
qty1Coefficients = [1, -1.25]
qty1Dict = {}
qty2Coefficients = [-1.25, 1]
qty2Dict = {}

for u in range(2):
  for c in range(200):
    costDict[(u, c)] = costCoefficients[u]
    capacityDict[(u, c)] = capacityCoefficients[u]
    lengthDict[(u, c)] = lengthCoefficients[u]
    qty1Dict[(u, c)] = qty1Coefficients[u]
    qty2Dict[(u, c)] = qty2Coefficients[u]

In [ ]:
# Defining Objective Function
m1.setObjective(N.prod(costDict))

# Generating Constraints
m1.addConstrs(N.prod(capacityDict, '*', c) >= demand[c]  for c in range(200))
m1.addConstr(N.prod(qty1Dict) <= 0)
m1.addConstr(N.prod(qty2Dict) <= 0)
m1.addConstrs(N.prod(lengthDict, '*', c) <= lengthRequirement[c] for c in range(200))
m1.addConstrs(N[u, c] >= 0 for u in range(2) for c in range(200))

m1.optimize()

In [ ]:
solution = N
OC = 0
OH = 0
for index in solution.keys():
  if index[0] == 0 and solution[index].X != 0:
    OC += solution[index].X
  if index[0] == 1 and solution[index].X != 0:
    OH += solution[index].X


print(OC)
print(OH)

In [ ]:
# Solve LP relaxation
r1 = m1.relax()
r1.optimize()

Exercise 3 - Formulation 2 (Compositions + Cross-sections)

In [ ]:
# Create set of compositions - denoted as a pair of values (OC, OH), where OC is the number of OC rolling stock units and OH is the number of OH rolling stock units
compositions = []
minDemand = min(demand)

for OC in range(4):
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

print(compositions)

In [ ]:
# Create model
m2 = gp.Model("second formulation ex 3")

# create variables
X = m2.addVars(200, 10, vtype=GRB.BINARY) #X_{c,p} variables (200 cross-sections & 10 compositions)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
lengthCoefficients = [100, 70]
lengthDict2 = {}
qty1Coefficients = [1, -1.25]
qty1Dict2 = {}
qty2Coefficients = [-1.25, 1]
qty2Dict2 = {}

for composition in range(len(compositions)):
  for c in range(200):
    costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
    capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
    lengthDict2[(c, composition)] = lengthCoefficients[0] * compositions[composition][0] + lengthCoefficients[1] * compositions[composition][1]
    qty1Dict2[(c, composition)] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
    qty2Dict2[(c, composition)] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

In [ ]:
# Defining Objective Function
m2.setObjective(X.prod(costDict2))

# Generating Constraints
m2.addConstrs(X.prod(capacityDict2, c, '*') >= demand[c]  for c in range(200))
m2.addConstr(X.prod(qty1Dict2) <= 0)
m2.addConstr(X.prod(qty2Dict2) <= 0)
m2.addConstrs(X.prod(lengthDict2, c, '*') <= lengthRequirement[c] for c in range(200))
m2.addConstrs(X.sum(c, '*') == 1 for c in range(200))

m2.optimize()

In [ ]:
solution = m2.getVars()
solution = solution[:2000]
shortSolutions = []
compositionsSolution = [0 for x in range(10)]
typesSolution = [0 for x in range(2)] # (OC, OH)

for variable in solution:
  if variable.X >= 0.8:
    shortSolutions.append(variable)

for variable in shortSolutions:
  compositionsSolution[variable.index % 10] += 1

for c in range(10):
  typesSolution[0] += compositions[c][0] * compositionsSolution[c]
  typesSolution[1] += compositions[c][1] * compositionsSolution[c]

print(typesSolution)

In [ ]:
# Solve LP relaxation
r2 = m2.relax()
r2.optimize()

**Exercise** 4 - Stochastic Demand

In [ ]:
#Assignment 4 third first trial
compositions = []
indexes = []
minDemand = min(demand)

for OC in range(4): #indexes 2, 3, 6, 8, 9
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

for c in range(200):
  for p in range(10):
      if line[c] != 400 or p in [0, 1, 4, 5, 7]:
        indexes.append((c,p))

costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
qty1Coefficients = [1, -1.25]
qty1Dict2 = {}
qty2Coefficients = [-1.25, 1]
qty2Dict2 = {}

for index in range(len(indexes)):
  c = indexes[index][0]
  composition= indexes[index][1]
  costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
  capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
  qty1Dict2[(c, composition)] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
  qty2Dict2[(c, composition)] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

Probability = {}

for index in indexes:
  mean = demand[index[0]]
  stdv = demand_stdDev[index[0]]
  Probability[index] = norm.cdf(capacityDict2[index], mean, stdv)

In [ ]:
# Create model
m3 = gp.Model("Assignment 4")

#variables
X3 = m3.addVars(indexes, vtype = GRB.BINARY, name = "X3")

In [ ]:
# Defining Objective Function
m3.setObjective(X3.prod(costDict2), GRB.MINIMIZE)

# Generating Constraints
m3.addConstr(X3.prod(Probability) >= 162)
m3.addConstrs(X3.prod(Probability, c, "*") >= 0.5 for c in range(200))
m3.addConstr(X3.prod(qty1Dict2) <= 0)
m3.addConstr(X3.prod(qty2Dict2) <= 0)
m3.addConstrs(X3.sum(c, '*') == 1 for c in range(200))

m3.optimize()

In [ ]:
#rolling stocks
solution = X3
OCUsed = 0
OHUsed = 0

print(solution)

for index in solution.keys():
  if solution[index].X == 1:
    OCUsed += compositions[index[1]][0]
    OHUsed += compositions[index[1]][1]

print(OCUsed)
print(OHUsed)

**Exercise** 5 - varying demand

In [ ]:
#Assignment 5
compositions = []
indexes = []
minDemand = min(demand)

for OC in range(4): #indexes 2, 3, 6, 8, 9
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

print(compositions)

for c in range(200):
  for p in range(10):
      if line[c] != 400 or p in [0, 1, 4, 5, 7]:
        indexes.append((c,p))

costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
qty1Coefficients = [1, -1.25]
OCunits = {}
qty2Coefficients = [-1.25, 1]
OHunits = {}

for index in range(len(indexes)):
  c = indexes[index][0]
  composition = indexes[index][1]
  costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
  capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
  OCunits[(c, composition)] = compositions[composition][0]
  OHunits[(c, composition)] = compositions[composition][1]

ProbabilityH = {}
ProbabilityL = {}

for index in indexes:
  mean = demand[index[0]]
  stdv = demand_stdDev[index[0]]
  ProbabilityH[index] = norm.cdf(capacityDict2[index], mean * 1.15, stdv)
  ProbabilityL[index] = norm.cdf(capacityDict2[index], mean * 0.9, stdv)

print(ProbabilityH )

In [ ]:
# Create model
m4 = gp.Model("Assignment 5", env = env)

#variables
XH = m4.addVars(indexes, vtype = GRB.BINARY, name = "XH")
XL = m4.addVars(indexes, vtype = GRB.BINARY, name = "XL")

XBench = m4.addVars(indexes, vtype = GRB.BINARY, name = "XBench")

In [ ]:
#Benchmark weighted average
# Defining  Objective Function
m4.setObjective(0.4 * XBench.prod(ProbabilityH) + 0.6 * XBench.prod(ProbabilityL), GRB.MAXIMIZE)

#Generating Constraints
m4.addConstr(XBench.prod(OCunits) == OCUsed)
m4.addConstr(XBench.prod(OHunits) == OHUsed)
m4.addConstrs(XBench.sum(c, '*') == 1 for c in range(200))

m4.optimize()

In [ ]:
solution = XBench

objValL = 0
objValH = 0
for index in solution.keys():
  if solution[index].X == 1:
    objValH += ProbabilityH[index]
    objValL += ProbabilityL[index]
print(objValH)
print(objValL)

In [ ]:
# actual model high demand
# Defining Objective Function
m4.setObjective(XH.prod(ProbabilityH), GRB.MAXIMIZE)

# Generating Constraints
m4.addConstr(XL.prod(OCunits) <= OCUsed)
m4.addConstr(XH.prod(OCunits) <= OCUsed)
m4.addConstr(XL.prod(OHunits) <= OHUsed)
m4.addConstr(XH.prod(OHunits) <= OHUsed)
m4.addConstrs(XH.sum(c, '*') == 1 for c in range(200))
m4.addConstrs(XL.sum(c, '*') == 1 for c in range(200))

m4.optimize()

In [ ]:
# actual model low demand
# Defining Objective Function
m4.setObjective(XL.prod(ProbabilityL), GRB.MAXIMIZE)

# Generating Constraints
m4.addConstr(XL.prod(OCunits) <= OCUsed)
m4.addConstr(XH.prod(OCunits) <= OCUsed)
m4.addConstr(XL.prod(OHunits) <= OHUsed)
m4.addConstr(XH.prod(OHunits) <= OHUsed)
m4.addConstrs(XH.sum(c, '*') == 1 for c in range(200))
m4.addConstrs(XL.sum(c, '*') == 1 for c in range(200))

m4.optimize()

In [ ]:
#actual model weighted average
# Defining Objective Function
m4.setObjective(0.4 * XH.prod(ProbabilityH) + 0.6 * XL.prod(ProbabilityL), GRB.MAXIMIZE)

# Generating Constraints
m4.addConstr(XL.prod(OCunits) <= OCUsed)
m4.addConstr(XH.prod(OCunits) <= OCUsed)
m4.addConstr(XL.prod(OHunits) <= OHUsed)
m4.addConstr(XH.prod(OHunits) <= OHUsed)
m4.addConstrs(XH.sum(c, '*') == 1 for c in range(200))
m4.addConstrs(XL.sum(c, '*') == 1 for c in range(200))

m4.optimize()

In [ ]:
#rolling stocks
solutionL = XL
solutionH = XH
solutionBench = XBench
OCUsedL = 0
OHUsedL = 0
OCUsedH = 0
OHUsedH = 0
OCUsedBench = 0
OHUsedBench = 0

for index in solutionBench.keys():
  if solutionL[index].X == 1:
    OCUsedL += compositions[index[1]][0]
    OHUsedL += compositions[index[1]][1]
  if solutionH[index].X == 1:
    OCUsedH += compositions[index[1]][0]
    OHUsedH += compositions[index[1]][1]
  if solutionBench[index].X == 1:
    OCUsedBench += compositions[index[1]][0]
    OHUsedBench += compositions[index[1]][1]

print(OCUsedL)
print(OHUsedL)
print(OCUsedH)
print(OHUsedH)
print(OCUsedBench)
print(OHUsedBench)

In [ ]:
initial_value_average = 161
final_value_average = 166
initial_value_H = 148
initial_value_L = 170
final_value_H = 151
final_value_L = 176

improvement_avg = ((final_value_average - initial_value_average) / initial_value_average) * 100
improvement_H = ((final_value_H - initial_value_H) / initial_value_H) * 100
improvement_L = ((final_value_L - initial_value_L) / initial_value_L) * 100

print("The improvement over the bench mark is on average:", improvement_avg, "%")
print("The improvement over the bench mark is on high demand days:", improvement_H, "%")
print("The improvement over the bench mark is on low demand days:", improvement_L, "%")

**Extension 1: Sensitivity**

In [ ]:
#Extension1: sensitivity analysis
compositions = []
indexes = []
minDemand = min(demand)
solMat = np.zeros((5, 5))

for OC in range(4): #indexes 2, 3, 6, 8, 9
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

print(compositions)

for c in range(200):
  for p in range(10):
      if line[c] != 400 or p in [0, 1, 4, 5, 7]:
        indexes.append((c,p))

costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
qty1Coefficients = [1, -1.25]
OCunits = {}
qty2Coefficients = [-1.25, 1]
OHunits = {}

for index in range(len(indexes)):
  c = indexes[index][0]
  composition= indexes[index][1]
  costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
  capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
  OCunits[(c, composition)] = compositions[composition][0]
  OHunits[(c, composition)] = compositions[composition][1]

listOfProbH = [0 for c in range(5)]
listOfProbL = [0 for c in range(5)]
highProb = [1.09, 1.12, 1.15, 1.18, 1.21]
lowProb = [0.86, 0.88, 0.9, 0.92, 0.94]
for h in range(5):
  ProbabilityH = {}
  for index in indexes:
    mean = demand[index[0]]
    stdv = demand_stdDev[index[0]]
    ProbabilityH[index] = norm.cdf(capacityDict2[index], mean * highProb[h], stdv)
  listOfProbH[h] = ProbabilityH
for l in range(5):
  ProbabilityL = {}
  for index in indexes:
    mean = demand[index[0]]
    stdv = demand_stdDev[index[0]]
    ProbabilityL[index] = norm.cdf(capacityDict2[index], mean * lowProb[l], stdv)
  listOfProbL[l] = ProbabilityL


In [ ]:
for h in range(5):
  for l in range(5):
    # Create model
    m5 = gp.Model("Assignment Sensitivity", env = env)

    #variables
    XHS = m5.addVars(indexes, vtype = GRB.BINARY, name = "XH")
    XLS = m5.addVars(indexes, vtype = GRB.BINARY, name = "XL")

    # Defining Objective Function
    m5.setObjective(0.4 * XHS.prod(listOfProbH[h]) + 0.6 * XLS.prod(listOfProbL[l]), GRB.MAXIMIZE)

    # Generating Constraints
    m5.addConstr(XLS.prod(OCunits) <= OCUsed)
    m5.addConstr(XHS.prod(OCunits) <= OCUsed)
    m5.addConstr(XLS.prod(OHunits) <= OHUsed)
    m5.addConstr(XHS.prod(OHunits) <= OHUsed)
    m5.addConstrs(XHS.sum(c, '*') == 1 for c in range(200))
    m5.addConstrs(XLS.sum(c, '*') == 1 for c in range(200))

    m5.optimize()
    solMat[l][h] = m5.objVal


In [ ]:
solMatR =np.floor(solMat).astype(int)
print(solMatR)

In [ ]:
plt.figure()
sns.heatmap(solMatR.astype(int), cmap='vlag', annot=True, fmt='.0f', cbar=True, xticklabels=highProb, yticklabels=lowProb) # create seaborn heatmap with annotations, xticklabels and yticklabels
# add a title
plt.title("Number of under-crowded trains")
# add axis labels
plt.ylabel("low demand peak")
plt.xlabel("high demand peak")
plt.show()

**Extension 2: Allowing for storage of rolling stock units between trips**

In [ ]:
# Data Preprocessing (Coefficients & Indexes)
# Generate Model
extension2Model = gp.Model("Extension 2 Model", env = env)

# Create coefficients (index 0 --> OC, index 1 --> OH)
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict = {}
capacityCoefficients = [620, 420]
capacityDict = {}
lengthCoefficients = [100, 70]
lengthDict = {}
qty1Coefficients = [1, -1.25]
qty1Dict = {}
qty2Coefficients = [-1.25, 1]
qty2Dict = {}

for u in range(2):
  for c in range(200):
    costDict[(u, c)] = costCoefficients[u]
    capacityDict[(u, c)] = capacityCoefficients[u]
    lengthDict[(u, c)] = lengthCoefficients[u]
    qty1Dict[(u, c)] = qty1Coefficients[u]
    qty2Dict[(u, c)] = qty2Coefficients[u]

# Generate Cofficients
indexes = []

for c in range(200):
  for u in range(1):
    indexes.append(((departureTime[c], arrivalTime[c]), (departureStation[c], arrivalStation[c]))) # ((t_D, t_A), (s_D, s_A))

N = extension2Model.addVars(2, 200, vtype = GRB.INTEGER, name = "N")
Y = extension2Model.addVars(2, 200, vtype = GRB.INTEGER, name = "Y")

In [ ]:
# Generate constraints to relate N to Y variables
extension2Model.setObjective(Y.prod(costDict))

for u in range(2):
  for c_star in range(200):
    constraint = LinExpr()
    for c in range(200):
      if indexes[c][1][1] == indexes[c_star][1][0] and indexes[c][0][1] < indexes[c_star][0][0]:
        constraint.add(N[u, c])
      elif indexes[c][1][0] == indexes[c_star][1][0] and indexes[c][0][0] < indexes[c_star][0][0]:
        constraint.add(Y[u, c] - N[u, c])

    constraint.add(Y[u, c_star])
    extension2Model.addConstr(constraint >= N[u, c_star])

extension2Model.addConstrs(N.prod(capacityDict, '*', c) >= demand[c]  for c in range(200))
extension2Model.addConstr(N.prod(qty1Dict) <= 0)
extension2Model.addConstr(N.prod(qty2Dict) <= 0)
extension2Model.addConstrs(N.prod(lengthDict, '*', c) <= lengthRequirement[c] for c in range(200))
extension2Model.addConstrs(N[u, c] >= 0 for u in range(2) for c in range(200))
extension2Model.addConstrs(Y[u, c] >= 0 for u in range(2) for c in range(200))

extension2Model.optimize()

In [ ]:
solution = Y
OC = 0
OH = 0
for index in solution.keys():
  if index[0] == 0 and solution[index].X != 0:
    OC += solution[index].X
  if index[0] == 1 and solution[index].X != 0:
    OH += solution[index].X


print(OC)
print(OH)